In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES']="0"
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import os
%load_ext autoreload
%autoreload 2
DATA_DIR='data_nuclis'
DATA_DIR_DEV='data_nuclis_dev'
#test_file='2tg_E98p_211a001{0}.png'
from image_utils import *


object_detection.utils not imported


In [33]:
generate_images_folder(DATA_DIR)
generate_images_folder(DATA_DIR_DEV)

object_detection.utils not imported
folder data_nuclis/2tg_E103p_223a001
folder data_nuclis_dev/2tg_E98p_224a001


In [37]:
get_py_px('cl1_2tg_E103p_223a001_00000_12_0_0_0.png')

object_detection.utils not imported


['12', '0']

In [21]:
generate_images_folder_v2(DATA_DIR)
#generate_images_folder_v2(DATA_DIR_DEV)

df data_nuclis/2tg_E103p_223a001/2tg_E103p_223a001{0}.png
folder data_nuclis/2tg_E103p_223a001 0 0
folder data_nuclis/2tg_E103p_223a001 0 1
folder data_nuclis/2tg_E103p_223a001 1 0
folder data_nuclis/2tg_E103p_223a001 1 1
df data_nuclis/2tg_E98p_224a001/2tg_E98p_224a001{0}.png
folder data_nuclis/2tg_E98p_224a001 0 0
folder data_nuclis/2tg_E98p_224a001 0 1
folder data_nuclis/2tg_E98p_224a001 1 0
folder data_nuclis/2tg_E98p_224a001 1 1


In [2]:
X_train,Y_train,file_names_train,PY_train,PX_train=get_images_data(DATA_DIR)
X_dev,Y_dev,file_names_dev,PY_dev,PX_dev=get_images_data(DATA_DIR_DEV)

In [43]:
PY_dev.shape

(1, 7424)

In [ ]:
import tensorflow as tf
DEV_FILE='data_nuclis_dev/2tg_E98p_224a001/2tg_E98p_224a001.png'
TRAIN_FILE='data_nuclis/2tg_E103p_223a001/2tg_E103p_223a001.png'

nx=X_train.shape[0]
num_classes=1
X = tf.placeholder(tf.float32, [nx,None],name="X")
W = tf.Variable(tf.zeros([num_classes, nx]),name="W")
b = tf.Variable(tf.zeros([num_classes,1]),name="b")
Z = tf.matmul(W, X) + b
Ypred=tf.sigmoid(Z)
Y = tf.placeholder(tf.float32, [num_classes, None])
cost=tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits = tf.transpose(Z),  labels = tf.transpose(Y)))#+tf.nn.l2_loss(W)
train_step = tf.train.GradientDescentOptimizer(0.001).minimize(cost)
correct_prediction = tf.equal(Y, tf.cast(tf.greater(Ypred,0.5),"float"))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
patches_dev,slices_dev=patches_of_image(DEV_FILE,40,40)
patches_train,slices_train=patches_of_image(TRAIN_FILE,40,40)
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    for i in range(100):
        _,j,a=sess.run([train_step,cost,accuracy],feed_dict={X:X_train,Y:Y_train})
        print i,"train",j,a
        print i,"test",sess.run([cost,accuracy],feed_dict={X:X_dev,Y:Y_dev})
    Ydev=sess.run(Ypred,feed_dict={X:patches_dev})
    Ytrain=sess.run(Ypred,feed_dict={X:patches_train})
    draw_rectangles(DEV_FILE,filter_rectangles(slices_dev[Ydev>=0.5],10))
    draw_rectangles(TRAIN_FILE,filter_rectangles(slices_train[Ytrain>=0.5],10))

In [3]:
import tensorflow as tf
DEV_FILE='data_nuclis_dev/2tg_E98p_224a001/2tg_E98p_224a001.png'
TRAIN_FILE='data_nuclis/2tg_E103p_223a001/2tg_E103p_223a001.png'
NUM_EPOCHS=3000

nx=X_train.shape[0]
num_classes=1
num_hidden1=4000
num_hidden2=500
with tf.variable_scope('test02') as scope:
    X = tf.placeholder(tf.float32, [nx,None],name="X")
    keep_prob = tf.placeholder(tf.float32)
    W1 = tf.get_variable("W1", [num_hidden1,nx], initializer = tf.contrib.layers.xavier_initializer(seed = 1),dtype=tf.float32)
    b1 = tf.get_variable("b1",dtype=tf.float32,initializer=tf.constant(np.zeros((num_hidden1,1)),dtype=tf.float32))
    W2 = tf.get_variable("W2", [num_hidden2,num_hidden1], initializer = tf.contrib.layers.xavier_initializer(seed = 1),dtype=tf.float32)
    b2 = tf.get_variable("b2",dtype=tf.float32,initializer=tf.constant(np.zeros((num_hidden2,1)),dtype=tf.float32))
    W3 = tf.get_variable("W3", [num_classes,num_hidden2], initializer = tf.contrib.layers.xavier_initializer(seed = 1),dtype=tf.float32)
    b3 = tf.get_variable("b3",dtype=tf.float32,initializer=tf.constant(np.zeros((num_classes,1)),dtype=tf.float32))
    Z1 = tf.matmul(W1, X) + b1
    A1=tf.nn.dropout(tf.nn.relu(Z1),keep_prob)
    Z2 = tf.matmul(W2, A1) + b2
    A2=tf.nn.dropout(tf.nn.relu(Z2),keep_prob)
    Z3 = tf.matmul(W3,A2)+b3
    Ypred=tf.sigmoid(Z3)
    Y = tf.placeholder(tf.float32, [num_classes, None])
    regul=tf.nn.l2_loss(W1)+tf.nn.l2_loss(W2)+tf.nn.l2_loss(W3)
    cost=tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits = tf.transpose(Z3),  labels = tf.transpose(Y)))+0.003*regul
    #train_step = tf.train.GradientDescentOptimizer(0.001).minimize(cost)
    train_step = tf.train.AdamOptimizer(0.00001).minimize(cost)
    correct_prediction = tf.equal(Y, tf.cast(tf.greater(Ypred,0.5),"float"))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
patches_dev,slices_dev=patches_of_image(DEV_FILE,40,40)
patches_train,slices_train=patches_of_image(TRAIN_FILE,40,40)
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    for i in range(NUM_EPOCHS):
        _,j,a=sess.run([train_step,cost,accuracy],feed_dict={X:X_train,Y:Y_train,keep_prob:0.5})
        if i%100==0:
            print i,"train",j,a
            print i,"test",sess.run([cost,accuracy],feed_dict={X:X_dev,Y:Y_dev,keep_prob:1.})
            Ydev=sess.run(Ypred,feed_dict={X:patches_dev,keep_prob:1.})
            Ytrain=sess.run(Ypred,feed_dict={X:patches_train,keep_prob:1.})
            draw_rectangles(DEV_FILE,filter_rectangles(slices_dev[Ydev>=0.5],30))
            draw_rectangles(TRAIN_FILE,filter_rectangles(slices_train[Ytrain>=0.5],20))
        #if i==2000:
        #    print "Canviant train_step"
        #    train_step=tf.train.GradientDescentOptimizer(0.0001).minimize(cost)

0 train 8.86087 0.489089
0 test [8.5396442, 0.6982758]
100 train 7.73098 0.910271
100 test [7.7137432, 0.90660912]
200 train 7.23465 0.942814
200 test [7.3099766, 0.90708804]
300 train 6.82512 0.946012
300 test [6.9223347, 0.90996158]
400 train 6.44129 0.948269
400 test [6.556386, 0.90660906]
500 train 6.07961 0.951467
500 test [6.2025352, 0.91187727]
600 train 5.73843 0.956922
600 test [5.8697162, 0.91618764]
700 train 5.41629 0.959932
700 test [5.5567245, 0.92073745]
800 train 5.11234 0.965199
800 test [5.262063, 0.9243294]
900 train 4.82514 0.970843
900 test [4.9917488, 0.92768186]
1000 train 4.557 0.975545
1000 test [4.7285337, 0.93199223]
1100 train 4.30599 0.975922
1100 test [4.4800267, 0.9363026]
1200 train 4.07018 0.978179
1200 test [4.2670331, 0.93103439]
1300 train 3.84625 0.983634
1300 test [4.042295, 0.93582368]
1400 train 3.63626 0.983634
1400 test [3.844888, 0.9370209]
1500 train 3.4418 0.985892
1500 test [3.6526983, 0.93941557]
1600 train 3.25677 0.987396
1600 test [3.46

In [13]:
s=slice(3,5)
range(10)[s]

[3, 4]

In [ ]:
from train import train4
train4(X_train.T,Y_train.T,X_dev.T,Y_dev.T,optimizer=tf.train.AdamOptimizer(0.0001))

0 train 4.23103 0.544
0 test [5.2074041, 0.29750001]
0 train 5.79466 0.46
0 test [1.5631038, 0.29949999]
0 train 4.38226 0.492
0 test [1.2459917, 0.70700008]
0 train 3.9786 0.5315
0 test [2.32796, 0.67696631]
10 train 1.19842 0.6775
10 test [0.45797372, 0.81700003]
10 train 1.10059 0.7
10 test [0.50069457, 0.80800009]
10 train 1.11647 0.6925
10 test [0.49471182, 0.79949999]
10 train 1.08571 0.697
10 test [0.46281713, 0.79915726]
20 train 0.507054 0.8065
20 test [0.42131779, 0.83050007]
20 train 0.503734 0.8215
20 test [0.47448111, 0.82250005]
20 train 0.480504 0.8275
20 test [0.45398396, 0.82050002]
20 train 0.485451 0.8105
20 test [0.41192424, 0.83146071]
30 train 0.346571 0.8605
30 test [0.31500667, 0.87150002]
30 train 0.337517 0.8605
30 test [0.35191435, 0.85550004]
30 train 0.311142 0.88
30 test [0.33761498, 0.85050005]
30 train 0.331322 0.8705
30 test [0.31598735, 0.85955065]
40 train 0.265413 0.8975
40 test [0.27623779, 0.88700008]
40 train 0.279707 0.887
40 test [0.30596223, 0.

350 train 0.0819822 0.966
350 test [0.20809877, 0.93800008]
350 train 0.0743508 0.976
350 test [0.20179132, 0.93100005]
350 train 0.081289 0.9685
350 test [0.1941248, 0.93328655]
360 train 0.0951835 0.958
360 test [0.16297194, 0.94000006]
360 train 0.0803724 0.967
360 test [0.21009, 0.93700004]
360 train 0.077547 0.9715
360 test [0.19598758, 0.93250012]
360 train 0.0811883 0.9705
360 test [0.19738139, 0.93469101]
370 train 0.0894623 0.9625
370 test [0.16119851, 0.94]
370 train 0.0809836 0.9695
370 test [0.20590386, 0.93950003]
370 train 0.0730269 0.9755
370 test [0.19640079, 0.93100005]
370 train 0.0775307 0.9725
370 test [0.19334994, 0.93398881]
380 train 0.0881126 0.964
380 test [0.16632794, 0.93950003]
380 train 0.0783142 0.972
380 test [0.20900202, 0.94]
380 train 0.0694259 0.9745
380 test [0.19635086, 0.93400007]
380 train 0.0732035 0.976
380 test [0.19117963, 0.93539333]
390 train 0.0889916 0.9655
390 test [0.16779624, 0.93900007]
390 train 0.0723625 0.9715
390 test [0.20911562, 

690 train 0.0315271 0.9925
690 test [0.21123561, 0.94030899]
700 train 0.0432296 0.988
700 test [0.16692881, 0.95050001]
700 train 0.0296996 0.995
700 test [0.23876613, 0.94400001]
700 train 0.030954 0.993
700 test [0.22501951, 0.93699998]
700 train 0.0310164 0.9925
700 test [0.21714732, 0.93960673]
710 train 0.0412602 0.987
710 test [0.18204781, 0.94400007]
710 train 0.0348026 0.9895
710 test [0.23804158, 0.94300008]
710 train 0.0302824 0.9935
710 test [0.22089228, 0.93850005]
710 train 0.0371331 0.9895
710 test [0.21182936, 0.94101119]
720 train 0.0372818 0.9915
720 test [0.16740513, 0.95000005]
720 train 0.0306235 0.9925
720 test [0.23908462, 0.94000006]
720 train 0.0300297 0.9945
720 test [0.22934069, 0.93699998]
720 train 0.0307317 0.99
720 test [0.21449521, 0.93960673]
730 train 0.0349291 0.9895
730 test [0.16911948, 0.94700003]
730 train 0.0306318 0.993
730 test [0.23734541, 0.94]
730 train 0.0304192 0.9915
730 test [0.22308342, 0.93900007]
730 train 0.0269021 0.993
730 test [0.

1030 train 0.0112788 0.9975
1030 test [0.24515051, 0.94241571]
1040 train 0.0137222 0.9975
1040 test [0.19530563, 0.95200002]
1040 train 0.0127815 0.9965
1040 test [0.28309253, 0.94050002]
1040 train 0.012451 0.9985
1040 test [0.26597035, 0.94000006]
1040 train 0.0118619 0.9965
1040 test [0.24592513, 0.93960673]
1050 train 0.013088 0.9965
1050 test [0.20603298, 0.9485001]
1050 train 0.0120271 0.997
1050 test [0.28811258, 0.94300008]
1050 train 0.0120328 0.998
1050 test [0.26425529, 0.93950003]
1050 train 0.0122022 0.9965
1050 test [0.24805617, 0.94241571]
1060 train 0.0135962 0.997
1060 test [0.1961292, 0.95550001]
1060 train 0.0153393 0.997
1060 test [0.28704679, 0.94200003]
1060 train 0.0110382 0.9995
1060 test [0.26508561, 0.9410001]
1060 train 0.0126228 0.9975
1060 test [0.24940789, 0.93960667]
1070 train 0.014056 0.997
1070 test [0.19460711, 0.95450008]
1070 train 0.0112715 0.998
1070 test [0.28246611, 0.94400001]
1070 train 0.0113697 0.999
1070 test [0.25791803, 0.94350004]
1070 

1360 train 0.00647662 0.9985
1360 test [0.29546246, 0.94241583]
1370 train 0.00790174 0.998
1370 test [0.23265333, 0.9515]
1370 train 0.00587252 0.9985
1370 test [0.33033961, 0.94350004]
1370 train 0.00547618 0.999
1370 test [0.30047685, 0.94200009]
1370 train 0.00505891 0.999
1370 test [0.28389502, 0.94382024]
1380 train 0.00709207 0.9985
1380 test [0.23837543, 0.9515]
1380 train 0.00404161 1.0
1380 test [0.33950162, 0.94149995]
1380 train 0.0048191 1.0
1380 test [0.30832604, 0.94150007]
1380 train 0.00492809 1.0
1380 test [0.29169166, 0.94382024]
1390 train 0.0125147 0.9965
1390 test [0.22457989, 0.95350003]
1390 train 0.00628837 0.9995
1390 test [0.34709179, 0.9460001]
1390 train 0.00710089 1.0
1390 test [0.30173329, 0.94350004]
1390 train 0.00465982 1.0
1390 test [0.29552025, 0.94101131]
1400 train 0.00592122 0.9995
1400 test [0.23264113, 0.95350003]
1400 train 0.00468872 0.999
1400 test [0.33255446, 0.94250011]
1400 train 0.00526362 0.9995
1400 test [0.31035614, 0.9410001]
1400 tr

...

5300 train 0.000560861 1.0
5300 test [0.54957652, 0.94800007]
5300 train 0.000911282 0.9995
5300 test [0.48682877, 0.94900006]
5300 train 0.000577413 0.9995
5300 test [0.4417212, 0.94943827]
5310 train 0.00123275 0.9995
5310 test [0.39481831, 0.95450002]
5310 train 0.0007587 0.9995
5310 test [0.54342079, 0.94900012]
5310 train 0.000445733 1.0
5310 test [0.4855186, 0.9485001]
5310 train 0.000320123 1.0
5310 test [0.43883306, 0.94733155]
5320 train 0.00129995 0.999
5320 test [0.4014228, 0.95200002]
5320 train 0.000697156 0.9995
5320 test [0.54535818, 0.94650006]
5320 train 0.000281217 1.0
5320 test [0.48818538, 0.94750005]
5320 train 0.00031276 1.0
5320 test [0.44223797, 0.94803375]
5330 train 0.000569218 1.0
5330 test [0.39629713, 0.954]
5330 train 0.000816951 1.0
5330 test [0.5520137, 0.94750011]
5330 train 0.000353302 1.0
5330 test [0.49213648, 0.94800013]
5330 train 0.000904766 0.9995
5330 test [0.44895762, 0.94803369]
5340 train 0.0011055 0.9995
5340 test [0.37690493, 0.95600003]
53

5630 train 0.000974547 0.9995
5630 test [0.40470481, 0.95200008]
5630 train 0.000703074 0.9995
5630 test [0.55543518, 0.94500011]
5630 train 0.000681394 0.9995
5630 test [0.49045503, 0.94400001]
5630 train 0.000885372 0.9995
5630 test [0.45591602, 0.94803369]
5640 train 0.00156741 0.999
5640 test [0.42209232, 0.95300007]
5640 train 0.000380825 1.0
5640 test [0.57335281, 0.94650006]
5640 train 0.000392701 1.0
5640 test [0.50631142, 0.94450009]
5640 train 0.00113465 0.9995
5640 test [0.46568999, 0.9438203]
5650 train 0.000694378 1.0
5650 test [0.39579883, 0.95550001]
5650 train 0.00123089 0.9995
5650 test [0.54374951, 0.94650006]
5650 train 0.000655512 0.9995
5650 test [0.4833743, 0.94750011]
5650 train 0.000628051 0.9995
5650 test [0.43460095, 0.94803369]
5660 train 0.000717502 1.0
5660 test [0.41497126, 0.95650005]
5660 train 0.000543625 0.9995
5660 test [0.56425285, 0.94750011]
5660 train 0.00110456 0.9995
5660 test [0.50293314, 0.94500005]
5660 train 0.000629911 0.9995
5660 test [0.4

5950 train 0.00104635 0.9995
5950 test [0.51806903, 0.94450003]
5950 train 0.000726932 0.9995
5950 test [0.4791891, 0.94803369]
5960 train 0.00121888 0.9995
5960 test [0.42972493, 0.95700008]
5960 train 0.00115272 0.999
5960 test [0.59753364, 0.949]
5960 train 0.000845254 0.9995
5960 test [0.53029752, 0.94850016]
5960 train 0.00033112 1.0
5960 test [0.48935479, 0.94733143]
5970 train 0.000868289 0.9995
5970 test [0.40777221, 0.954]
5970 train 0.00115781 0.999
5970 test [0.5611276, 0.94750011]
5970 train 0.000857362 0.9995
5970 test [0.49446398, 0.94400007]
5970 train 0.000462791 1.0
5970 test [0.46654725, 0.94733143]
5980 train 0.00125156 0.999
5980 test [0.41452765, 0.95549995]
5980 train 0.0010757 0.999
5980 test [0.57332814, 0.94749999]
5980 train 0.00027408 1.0
5980 test [0.50648892, 0.9460001]
5980 train 0.000668101 0.9995
5980 test [0.47763464, 0.94733143]
5990 train 0.000505435 1.0
5990 test [0.40851891, 0.95849997]
5990 train 0.000898693 0.9995
5990 test [0.57167029, 0.949]
599

6280 train 0.000973827 0.9995
6280 test [0.43070793, 0.95600003]
6280 train 0.00150383 0.999
6280 test [0.58555895, 0.94450009]
6280 train 0.000377231 1.0
6280 test [0.54030997, 0.93900007]
6280 train 0.00030869 1.0
6280 test [0.49187768, 0.9445225]
6290 train 0.00136898 0.999
6290 test [0.44957614, 0.95050001]
6290 train 0.000596098 1.0
6290 test [0.61463994, 0.94200003]
6290 train 0.000950291 0.9995
6290 test [0.56940484, 0.94000006]
6290 train 0.0012506 0.999
6290 test [0.51623023, 0.94452244]
6300 train 0.00171792 0.999
6300 test [0.44253016, 0.9525001]
6300 train 0.000415272 1.0
6300 test [0.61489409, 0.94300008]
6300 train 0.00083654 0.9995
6300 test [0.54790252, 0.94200003]
6300 train 0.000533746 0.9995
6300 test [0.50749815, 0.94733155]
6310 train 0.000446224 1.0
6310 test [0.42940146, 0.954]
6310 train 0.00114992 0.9995
6310 test [0.57436186, 0.94850004]
6310 train 0.00097889 0.9995
6310 test [0.53130084, 0.94600004]
6310 train 0.000573524 0.9995
6310 test [0.48885229, 0.94733

6600 train 0.000524006 1.0
6600 test [0.43985441, 0.94522476]
6610 train 0.00141696 0.999
6610 test [0.39293596, 0.95450002]
6610 train 0.00108267 0.9995
6610 test [0.55455267, 0.94900006]
6610 train 0.00079539 0.9995
6610 test [0.47607917, 0.94550008]
6610 train 0.00072949 1.0
6610 test [0.43680954, 0.94522476]
6620 train 0.00110357 0.9995
6620 test [0.39481306, 0.95450002]
6620 train 0.000720103 1.0
6620 test [0.55853736, 0.94700003]
6620 train 0.000833981 0.9995
6620 test [0.48113066, 0.94500011]
6620 train 0.000596851 1.0
6620 test [0.44170073, 0.94662929]
6630 train 0.00133363 0.999
6630 test [0.39702588, 0.95350003]
6630 train 0.000862659 0.9995
6630 test [0.55988967, 0.94750011]
6630 train 0.000663247 0.9995
6630 test [0.48212785, 0.94600016]
6630 train 0.00067672 0.9995
6630 test [0.44193336, 0.94592702]
6640 train 0.00140412 0.999
6640 test [0.39580041, 0.95400006]
6640 train 0.000804522 0.9995
6640 test [0.56396663, 0.94850004]
6640 train 0.000456814 1.0
6640 test [0.4862743,

In [7]:
X_train.shape

(4800, 9416)